In [130]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
import json
import requests
import os
#import Levenshtein as lev
mu.css()

In [133]:
AVSNITTMAPPE = "Heidis_avsnitt/"

In [132]:
#!pip install fuzzywuzzy

In [131]:
#!pip install python-Levenshtein

In [116]:
from fuzzywuzzy import fuzz

In [22]:
def get_paragraphs(urn, paras):
    """Return paragraphs for urn"""
    param = dict()
    param['paragraphs'] = paras
    param['urn'] = urn
    r = requests.get("https://api.nb.no/ngram/paragraphs", json=param)
    return r.json()

In [23]:
def get_para(para_list):
    import ast
    paras = []
    for p in para_list:
        paras.append(ast.literal_eval(p))
    urns = [x[0] for x in paras]
    paradict = {u:[x[1] for x in paras if x[0] == u] for u in urns}
    paragraphs = dict()
    for u in paradict:
        paragraphs[u] = get_paragraphs(u, paradict[u])
    return paragraphs


In [26]:
themes = json.load(open('heidis_temadefinisjoner.json'))

In [51]:
link = lambda x: "https://urn.nb.no/URN:NBN:no-nb_digibok_" + str(x)

In [31]:
for th in themes:
    print(th)

“Jul” [Christmas]
“Kvindens funksjon i den private sfære” [Woman’s function in the private sphere]
“Kvindens funksjon i den offentlige sfære” [Woman’s function in the public sphere]
“Kvinnelig dannelse” [Woman’s bildung]
“Kvinnens væren” [Woman’s being]
“Er kjærligheten sann” [Is love true?]
“Kvinnens beskyttelse” [Woman’s protection]
“Kvinnens mot under prøvelser” [Woman’s courage during difficult times]
“Båndet mellom kjønnene” [The bond between the genders]
“Slavinnen og herren” [(Female) Slave and master]
“Tjenerinnes romantiske forbindelse” [the Female servant’s romantic liaison]
“Kvinnes selvmord” [Woman’s suicide]
“Kvindens vilje” [Woman’s Volition]
“Innflytelse over kvinnen” [Influence on Woman]
“Et fruentimmers dyd” [Woman’s virtues]
“En kvinnes væren og tillærte egenskaper” [Woman’s being and acquired traits]
“Kvindens stillhetens språk” [Woman’s language of silence]
“Mill, en frigjører eller rabulist?” [Mill, a liberator or an offender]
“Kvindens likegyldighed til frihetsret

In [53]:
print(themes[find_theme('Jul', themes)])

{'Mor_': ['Mor', 'Mødre', 'moderlig', 'moderligt', 'Modcr', 'modcr', '', 'Modre', 'Msdre', 'Msdrc', 'modre', 'modcrlig', 'modrcne', 'modrc', 'modcrlig', 'modcrlighcden', 'modcrlighed', 'moderlighed', 'modcrlighcd', 'Huusmoder', 'husmoderlig', 'husmoderlige', 'husmoderlighed', 'husmoderligheden', 'Husmor', 'Husmoder', 'Husmodcr', 'Husmodcrcn', 'Husmoderen', 'Husmodcren', 'Husmodcren', 'Huusmodcr', 'Huusmodre', 'Huusmoders', 'Huusmodcr', 'Huusmodrene', 'Huusmodcren', 'Huusmodercn', 'Huusmodcrs', 'Husmodre', 'Husmodrc', 'Husmoderem', 'Husmoderms', 'Huusmodcrcn', 'Moderlig', 'Moderligt', 'Modcrlig', 'Modrcne', 'Modrc', 'Modcrlighcden', 'Modcrlighed', 'Moderlighed', 'Modcrlighcd', 'Husmoderlig', 'Husmoderlige', 'Husmoderlighed', 'Husmoderligheden'], 'Kvinde_': ['kvinne', 'kvinnen', 'kvinner', 'kvinnene', 'kvinnerne', 'kvinnes', 'kvinnens', 'kvinners', 'kvinnenes', 'kvinnernes', 'kvinde', 'Kvmde', 'kvinden', 'kvmden', 'kvinder', 'kvinderne', 'kvindens', 'kvmdens', 'kvindes', 'kvmdes', 'kvind

In [117]:
import os
r,d,f = next(os.walk("."))
jsons = [fs for fs in f if fs.endswith('.json')]
files = {f:find_theme(f, themes) for f in jsons if find_theme(f, themes) != ''}

In [118]:
files

{'Er kjærligheten sann Is love true.json': '“Er kjærligheten sann” [Is love true?]',
 'Jul Christmas.json': '“Jul” [Christmas]',
 'Kvindens funksjon i den offentlige sfære Womans function in the public sphere.json': '“Kvindens funksjon i den offentlige sfære” [Woman’s function in the public sphere]',
 'Kvindens funksjon i den private sfære Womans function in the private sphere.json': '“Kvindens funksjon i den private sfære” [Woman’s function in the private sphere]',
 'Kvinnelig dannelse Womans bildung.json': '“Kvinnelig dannelse” [Woman’s bildung]',
 'Kvinnens væren Womans being.json': '“Kvinnens væren” [Woman’s being]'}

In [101]:
def find_theme(title, themes):
    from collections import Counter
    dist = Counter()
    for th in themes:
        dist[th] = fuzz.ratio(th, title)
    x = dist.most_common(1)[0][0]
    if dist[x] > 60:
        res = x
    else:
        res = ""
    return res

In [112]:
def show_paras(title, themes):
    avsnitt = json.load(open(title))
    paras = get_para(avsnitt)
    theme = find_theme(title, themes)
    words = [w for hyperlexeme in themes[theme] for w in themes[theme][hyperlexeme]]
    ptext = "<h2>" + theme + "</h2>"
    ptext += "<h3" + ', '.join(themes[theme].keys()) + "</h3>"
    for urn in paras:
        ptext += "<h4> <a href='" + link(urn) +"'" + "target=_>" + ', '.join([str(x) for x in nb.metadata(urn)[0][1:4]]) + "</a>"
        for paranum in paras[urn]:
            ptext += "<h4>" + paranum + '</h4>'
            modified = paras[urn][paranum]
            #print(modified)
            for w in words:
                #print(w)
                modified = modified.replace(" "+w+" ", " <b>" + w + "</b> ")
            ptext += modified + '\n\n'
    return ptext

In [127]:
for f in files:
    print(f.split('.json')[0])

Er kjærligheten sann Is love true
Jul Christmas
Kvindens funksjon i den offentlige sfære Womans function in the public sphere
Kvindens funksjon i den private sfære Womans function in the private sphere
Kvinnelig dannelse Womans bildung
Kvinnens væren Womans being


In [134]:
res = ""
for f in files:
    print(f)
    newfilename = f.split('.json')[0] + '_avsnitt.html'
    res = show_paras(f, themes) + "\n"
    with open(os.path.join(AVSNITTMAPPE, newfilename), "w", encoding='utf-8') as html:
        html.write(res)

Er kjærligheten sann Is love true.json
Jul Christmas.json
Kvindens funksjon i den offentlige sfære Womans function in the public sphere.json
Kvindens funksjon i den private sfære Womans function in the private sphere.json
Kvinnelig dannelse Womans bildung.json
Kvinnens væren Womans being.json
